In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
from descartes import PolygonPatch
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
import json

%matplotlib inline
ox.config(log_console=True)
ox.__version__

'1.1.1'

## Objetos Propios

### Isochrone Plots

In [2]:
from networkx.classes.multidigraph import MultiDiGraph

class IsochronePlots:
    def __init__(self, trip_times: list):
        self.trip_times = trip_times
    
    def colors(self) -> list:
        self.colors = ox.plot.get_colors(
            n=len(self.trip_times),
            cmap="plasma",
            start=0,
            return_hex=True)
        return self.colors

    def plot_node_isochrones(self,G: MultiDiGraph ,center_node: int) -> None:
        node_colors = {}
        for trip_time, color in zip(sorted(self.trip_times, reverse=True), self.colors):
            subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance="time")
            for node in subgraph.nodes():
                node_colors[node] = color
        node_colors = [node_colors[node] if node in node_colors else "none" for node in G.nodes()]
        node_size = [15 if node in node_colors else 0 for node in G.nodes()]
        return node_colors,node_size
#         fig, ax = ox.plot_graph(
#             G,
#             node_color=nc,
#             node_size=ns,
#             node_alpha=0.8,
#             edge_linewidth=0.2,
#             edge_color="#999999",
#         )

## GraphBuilder

In [3]:
class GraphBuilder:
    def __init__(self, place, transportation_mode, travel_speed):
        self.place = place
        self.transportation_mode = transportation_mode
        self.travel_speed = travel_speed # in km/h
        self.graph = None

    def initialize_graph(self):
        self.graph = ox.graph_from_place(
            self.place,
            network_type=self.transportation_mode
        )
        return self.graph

    def calculate_centroid(self):
        gdf_nodes = ox.graph_to_gdfs(self.graph, edges=False)
        x, y = gdf_nodes["geometry"].unary_union.centroid.xy
        center_node = ox.distance.nearest_nodes(self.graph, x[0], y[0])
        return center_node
        
    def initialize_projected_graph(self):
        self.projected_graph = ox.project_graph(self.graph)
        meters_per_minute = self.travel_speed * 1000 / 60  # km per hour to m per minute
        
        for _, _, _, data in self.projected_graph.edges(data=True, keys=True):
            data["time"] = data["length"] / meters_per_minute
        
        return self.projected_graph

### IsochroneJsonDump

In [4]:
class IsochroneJsonDump:
    
    def __init__(self, node_colors, node_size):
        self.node_colors = node_colors
        self.node_size = node_size
        
    def serialize(self):
        return {"node_colors":self.node_colors,"node_size":self.node_size}

In [5]:
place = "Buenos Aires, Argentina"
network_type = "walk"

trip_times = [5, 10, 15, 30, 45, 60, 90]  # in minutes
travel_speed = 4.5  # walking speed in km/hour

graph_builder = GraphBuilder(place, network_type, travel_speed)
graph_builder.initialize_graph()

In [6]:
G = graph_builder.graph

In [7]:
center_node = graph_builder.calculate_centroid()
projected_G = graph_builder.initialize_projected_graph()

# get one color for each isochrone
isochrone_plots = IsochronePlots(trip_times)

iso_colors = isochrone_plots.colors()

In [8]:
print(G.size())
print(len(G.nodes))
print(len(G.edges))
with open("jsonDumpIsochrones.json", "w") as archivo:
    isochronesForAllNodes = {}
    for i in list(G.nodes)[:3]:
        node_colors, node_size = isochrone_plots.plot_node_isochrones(projected_G,i)
        isochrones = IsochroneJsonDump(node_colors,node_size)
        isochronesForAllNodes[i] = isochrones.serialize()
    archivo.write(json.dumps(isochronesForAllNodes))

113566
35692
113566


In [ ]:
#ox.distance.get_nearest_node() - Caro, ari me dijo que podiamos usar esto con las coordenadas de los parques
#para tomar de referencia ese nodo como el nodo del parque